<a href="https://colab.research.google.com/github/SanjanaA01/Projects/blob/main/RankingDocuments_Inverted_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -rf input_docs
!cp /content/drive/MyDrive/input_docs_sample.zip .
!unzip input_docs.zip > /dev/null
!ls input_docs/ | wc -l

unzip:  cannot find or open input_docs.zip, input_docs.zip.zip or input_docs.zip.ZIP.
ls: cannot access 'input_docs/': No such file or directory
0


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Create an RDD from a text file**

Each line of the text file becomes an element of the RDD.

In [ ]:
# wholeTextFiles generates an RDD of pair values, 
# where the key is the full path of each file, the value is the content of each file
#input = sc.wholeTextFiles("/content/drive/My\ Drive/input_docs");
input = sc.wholeTextFiles("input_docs");

# Now we strip the prefix of filenames and leave only the basename. 
# e.g. 'file:/content/drive/My Drive/Colab Notebooks/data_spark/input_docs/3.html'
# becomes '3.html' 
import os

#(did,text)
input2 = input.map(lambda x: (int(os.path.basename(x[0]).split(".")[0]), x[1]))

print(input2.take(1))

Py4JJavaError: ignored

In [ ]:
nltk.download('punkt')
from bs4 import BeautifulSoup
import collections
from collections import Counter
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stem = PorterStemmer()

In [ ]:
# Doc to wordlist function
# The output will be a list of tuples such as 
# ("apple", (3,10,10/20)), 
# where 3 is docid, 
# 10 is frequency of "apple" in this doc, 
# 20 is maxf in in this doc.

from bs4 import BeautifulSoup
from collections import Counter
import re

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def dw(docid, htmltext):

  htmltext = htmltext.lower() #convert into lowercase
  cleantext = BeautifulSoup(htmltext).get_text()
  
  cleantext = re.sub("\d+", " ", cleantext) #remove digits
  cleantext = re.sub('[^A-Za-z0-9]+', ' ', cleantext) #remove special characters
  cleantext = re.sub("\s+", " ", cleantext) #remove extra spaces
  
  cleantext = word_tokenize(cleantext)
  cleantext = [word for word in cleantext if word not in stopwords.words('english')]
  
  output = [];
  counter = Counter(cleantext)
  maxf = Counter(cleantext).most_common(1)[0][1]
  for ele in counter:
    tuple1 = (ele);
    tuple2 = (int(docid),counter[ele],counter[ele]/maxf)
    tuple3 = (tuple1, tuple2)
    output.append(tuple3);
  return(output)

word_docid_freq_tf = input2.flatMap(lambda x: dw(x[0],x[1]))
print(word_docid_freq_tf.take(2))

In [ ]:
%xmode Verbose
#combining all the words and documents ids frequency
word_list_freq_tf = word_docid_freq_tf.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x+y)
# Now create an RDD as follows 
# (word, [(did1,freq1,tf1), (did2,freq2,tf2), ...])
word_postinglist_freq_tf = sc.parallelize(word_list_freq_tf.collect())
print(word_postinglist_freq_tf.count())
print(word_postinglist_freq_tf.map(lambda x : (x[0], list(x[1]))).take(1))

In [ ]:
# idf = 1/len(postinglist_tf)
%xmode Verbose
%time
import math
from decimal import Decimal
with_tfidf = []
for row in word_postinglist_freq_tf.collect():
  idf = 1/len(row[1]) #present in number of documents
  backto_tuple = []
  print(row[1])
  for eachrow in row[1]:
    list_row = list(eachrow) #convert to list
    # list_row[2] = float(format(float(list_row[2]*idf), '.5f')) #tf*idf
    tfidf_val = Decimal((list_row[2]*idf)) #tf*idf
    list_row[2] ="{:.18f}".format(float(tfidf_val))
    backto_tuple.append(tuple(list_row))

  eachtuple = (row[0], backto_tuple)
  with_tfidf.append(eachtuple);
# print(with_tfidf[0])
# creating a RDD
word_postinglist_freq_tfidf = sc.parallelize(with_tfidf)

print(word_postinglist_freq_tfidf.take(2))
print(word_postinglist_freq_tfidf.count(2))


**Calculating Magnitude**




In [ ]:
# Now, we would like to obtain the magnitude of each doc.
# First, produce (did, (freq,tfidf)) for each word of doc did; 
# We do don't need the word itself, just its (freq,tfidf). 
# Then, do reduceByKey on these tuples and obtain maxfreq and 
# magnitude (squared) for each document. 
%xmode Verbose
import math
from decimal import Decimal

convert_to_list = word_postinglist_freq_tfidf.collect() #make a list
convert_to_list = list(map(list, convert_to_list)) 
new_list = []
for val in convert_to_list: 
  for row in val[1]:
    row = list(row)
    # print(format(Decimal.from_float(0.1), '.17'))
    # squareval = float(format((row[2]*row[2]), '.15'))
    # print(type(row[2]))
    magnitude = Decimal((float(row[2])**2))
    magnitude ="{:.18f}".format(float(magnitude))
    row = (row[0],(row[1],magnitude))
  new_list.append(row)

# RDD of (did,(freq,tfidf)) tuples
# creating a RDD
did_freq_tfidfsq_rdd = sc.parallelize(new_list)

print(did_freq_tfidfsq_rdd.take(2))
new_list1 = did_freq_tfidfsq_rdd.reduceByKey(lambda x,y: (max(x[0],y[0]),(float(x[1])+float(y[1]))))

# Produce (did,(maxf,magnitudesq))
# creating a RDD
doc_maxf_mag = sc.parallelize(new_list1.collect())

#print(doc_maxf_mag.count())
print(doc_maxf_mag.take(2))

In [ ]:
!rm -rf inv_idx
word_postinglist_freq_tfidf.saveAsTextFile("inv_idx");

In [ ]:
!rm -rf doc_mag
doc_maxf_mag.saveAsTextFile("doc_mag");

In [ ]:
!ls -lrt inv_idx
!head inv_idx/part-00001
!wc -l inv_idx/part-00000
!wc -l inv_idx/part-00001
!cat inv_idx/part-00000 inv_idx/part-00001 > /content/drive/My\ Drive/inv_idx.txt
!wc -l /content/drive/My\ Drive/inv_idx.txt

In [ ]:
!ls -lrt doc_mag
!head doc_mag/part-00000
!wc -l doc_mag/part-00000
!wc -l doc_mag/part-00001
!cat doc_mag/part-00000 doc_mag/part-00001 > /content/drive/My\ Drive/doc_mag.txt
!wc -l /content/drive/My\ Drive/doc_mag.txt